# Time Series - Exploratory Data Analysis

This notebook explores various time series data exploration techniques including visualization, decomposition, and anomaly detection. It also covers methods for handling high-frequency data with Fourier transforms and wavelet analysis. 

Throughout, we will be building a general EDA framework and tool that can be reused for various time series datasets.

Table of Contents:

0. [Data Loading and Preprocessing](#data-loading-and-preprocessing)
1. [The time index](#the-time-index)
2. [Measures of Distribution](#measures-of-distribution)
3. [Stationarity](#stationarity)
4. [Self-Correlation](#self-correlation) 
5. [Putting it All Together: An EDA Class](#putting-it-all-together-an-eda-class)

In [48]:
import pandas as pd
import plotly.graph_objects as go
import statsmodels
from plotly.subplots import make_subplots
from pathlib import Path

# Data Loading and Preprocessing

Before we get to anything else, we need to load some data. For this notebook, we will use some energy demand data from Kaggle, showing the electrical demand in several European in megawatt hours (MWh) at one hour intervals. Because it is at the top of the alphabet, we will look at Austria's demand data.

In [53]:
column = 'AT_load_actual_entsoe_transparency'
fp = Path('../data/forecasting/load-wind-and-solar-prices-in-hourly-resolution/time_series_60min_singleindex.csv')
df = pd.read_csv(fp)

# index manipulation - to datetime, set as index
df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'])
df = df.set_index('utc_timestamp')

# getting the Austria load column
df = df.filter([column])
df = df.rename(columns={column: 'load'})

# drop missing values
df = df.dropna()

We now have the data loaded into a Pandas DataFrame called `df` as a single column time series with a datetime index. Let's verify this by checking the first few rows of the DataFrame.

In [54]:
df.head()

,load
utc_timestamp,
2015-01-01 00:00:00+00:00,5946.0
2015-01-01 01:00:00+00:00,5726.0
2015-01-01 02:00:00+00:00,5347.0
2015-01-01 03:00:00+00:00,5249.0
2015-01-01 04:00:00+00:00,5309.0


# The time index

For any data series of temporally indexed data, making sense of the time index is crucial. Let's start by examining the time index of our DataFrame `df`. We want to understand the frequency of the data points, and the overall time span of the dataset. We also want to check for any missing timestamps that could affect our analysis - this is different from tabular data where we just check for missing values in the data columns themselves. Here, we know that if a timestamp is missing, it means we have no data for that time point.

In [21]:
def describe_time_index(df: pd.DataFrame) -> dict:
    """
    Describe the time index of a DataFrame in terms of frequency, time span, and missing timestamps.

    Parameters:
    df (pd.DataFrame): DataFrame with a datetime index.
    
    Returns:
    dict: A dictionary containing the inferred frequency, start time, end time, and missing timestamps
    of the time index.

    Raises:
    ValueError: If the DataFrame index is not a datetime type.
    """

    index = df.index
    if not pd.api.types.is_datetime64_any_dtype(index):
        raise ValueError("DataFrame index must be a datetime type.")

    # Frequency
    inferred_freq = pd.infer_freq(index)
    
    # Time span
    start_time = index.min()
    end_time = index.max()
    
    # Missing timestamps
    full_time_index = pd.date_range(start=start_time, end=end_time, freq=inferred_freq)
    missing_timestamps = full_time_index.difference(index)
    
    return {
        'inferred_frequency': inferred_freq,
        'start_time': start_time,
        'end_time': end_time,
        'n_missing_timestamps': len(missing_timestamps),
        'missing_timestamps': missing_timestamps.tolist()
    }

In [22]:
describe_time_index(df)

{'inferred_frequency': 'h',
 'start_time': Timestamp('2014-12-31 23:00:00+0000', tz='UTC'),
 'end_time': Timestamp('2020-09-30 23:00:00+0000', tz='UTC'),
 'n_missing_timestamps': 0,
 'missing_timestamps': []}

We can see that this data has hourly frequency with no missing timestamps. This removes concerns about irregular sampling or gaps in the data that could complicate our analysis later on.

The next step is to visualize the time series to get a sense of its overall structure and any obvious patterns or anomalies. This is the most intuitive step in time series EDA, as visual inspection can often reveal trends, seasonality, and outliers that statistical tests might miss.

In [32]:
def plot_time_series(df: pd.DataFrame):
    """
    Plot a univariate time series DataFrame using Plotly.

    Parameters:
    df (pd.DataFrame): DataFrame with a datetime index and a single column representing the time series data.

    Returns:
    fig: Plotly figure object displaying the time series.
    """

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['load'], mode='lines', name=df.columns[0]))
    fig.update_layout(title=df.columns[0] + ' time series',
                    xaxis_title='Time')
    fig.update_layout(template='plotly_white')

    return fig

In [33]:
fig = plot_time_series(df)
fig.show()

# Measures of Distribution

The next step in our time series exploratory data analysis is to examine the distribution of the data values themselves. Understanding the distribution helps us identify characteristics such as skewness, kurtosis, and the presence of outliers, which can inform our choice of modeling techniques later on. 

We make two histograms: one for the full dataset, and another for the differenced data to see how changes from one point in time to another are distributed. This can be very useful for identifying trends in the dataset. We also make quick calculations of key statistics like mean, median, standard deviation, skewness, and kurtosis to summarize the distribution.

Per *Practical Time Series Analysis*, p95:
> In a time series context, a (histogram) of the difference of the data is often more interest‐ ing than a (histogram)) of the untransformed data. After all, in a time series context, often (and particularly in finance) what is most interesting is how a value changes from one measurement to the next rather than the value’s actual measurement. This is particularly true for plotting, because taking the histogram of data with a trend in it does not produce a very informative visualization.

In [43]:
def describe_distribution(ts: pd.Series) -> dict:
    """
    Describe the distribution of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.

    Returns:
    dict: A dictionary containing key statistics of the distribution.
    """

    return {

        'min': ts.min(),
        '25th_percentile': ts.quantile(0.25),
        '50th_percentile': ts.quantile(0.50),
        '75th_percentile': ts.quantile(0.75),
        'max': ts.max(),
        'range': ts.max() - ts.min(),
        'mean': ts.mean(),
        'std_dev': ts.std(),
        'skewness': ts.skew(),
        'kurtosis': ts.kurtosis()
    }

def plot_distribution_histogram(ts: pd.Series) -> go.Figure:
    """
    Plot a histogram of the data.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.

    Returns:
    fig: Plotly figure object displaying the histogram.
    """

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=ts, nbinsx=50))
    fig.update_layout(title='Distribution of ' + ts.name,
                      xaxis_title=ts.name,
                      yaxis_title='Count')
    fig.update_layout(template='plotly_white')

    return fig

In [45]:
# regular histogram
distribution_stats = describe_distribution(df['load'])
distribution_histogram = plot_distribution_histogram(df['load'])

print(distribution_stats)
distribution_histogram.show()

{'min': np.float64(664.0), '25th_percentile': np.float64(5941.0), '50th_percentile': np.float64(7032.0), '75th_percentile': np.float64(8079.0), 'max': np.float64(10803.0), 'range': np.float64(10139.0), 'mean': np.float64(7070.148630952381), 'std_dev': np.float64(1400.2631669359246), 'skewness': np.float64(0.14004809936323945), 'kurtosis': np.float64(-0.5671520761708431)}


In [47]:
# differenced histogram and stats
diff_ts = df['load'].diff().dropna()
diff_distribution_stats = describe_distribution(diff_ts)
diff_distribution_histogram = plot_distribution_histogram(diff_ts)

print(diff_distribution_stats)
diff_distribution_histogram.show()

{'min': np.float64(-4451.0), '25th_percentile': np.float64(-227.0), '50th_percentile': np.float64(-63.0), '75th_percentile': np.float64(152.0), 'max': np.float64(5283.0), 'range': np.float64(9734.0), 'mean': np.float64(-0.010932756602313538), 'std_dev': np.float64(364.92299580636114), 'skewness': np.float64(0.9673402611826681), 'kurtosis': np.float64(3.495324363795505)}


# Stationarity

Stationarity can be a slippery concept to grasp, but it is crucial for many time series modeling techniques. A stationary time series has statistical properties, such as mean and variance, that do not change over time and allow us to expect that the system's long-term past behavior will continue to reflect in its future behavior. Many models, such as ARIMA, assume stationarity, so it's important to assess this property in our data before selecting a modeling approach.

The mathematical definition of stationarity can be complex, but for practical purposes, we can use statistical tests to assess whether our time series is stationary. One common test is the Augmented Dickey-Fuller (ADF) test, which tests the null hypothesis that a unit root is present in the time series. If we can reject this null hypothesis, we can conclude that the time series is stationary.

In [50]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(ts: pd.Series) -> dict:
    """
    Perform the Augmented Dickey-Fuller test to assess the stationarity of a time series.

    Parameters:
    ts (pd.Series): A pandas Series representing the time series data.

    Returns:
    dict: A dictionary containing the ADF test results.
    """

    adf_result = adfuller(ts)
    return {
        'adf_statistic': adf_result[0],
        'p_value': adf_result[1],
        'used_lag': adf_result[2],
        'n_obs': adf_result[3],
        'critical_values': adf_result[4],
        'ic_best': adf_result[5]
    }

In [55]:
test_stationarity(df['load'])

{'adf_statistic': np.float64(-16.67800764176947),
 'p_value': np.float64(1.524895598250865e-29),
 'used_lag': 57,
 'n_obs': 50342,
 'critical_values': {'1%': np.float64(-3.4304799041251957),
  '5%': np.float64(-2.8615974149640566),
  '10%': np.float64(-2.5668005600849857)},
 'ic_best': np.float64(648211.7023811574)}

# Self-Correlation

# Putting it All Together: An EDA Class